In [10]:
from sage.coding.code_bounds import entropy

def entropy_inv(y, q):
    if y < 0 or y > 1 :
        print("impossible")
        return -1
    if y == 0 :
        return 0
    if y == 1 :
        return 1 - 1/q
    x_pred = 0.5 - (1.0/(2.0*q))
    while entropy(x_pred, q) - x_pred * log((q - 1.0) * (1.0 - x_pred) / x_pred, q) > y :
        x_pred = x_pred / 2.0
    
    x = (y - entropy(x_pred, q))/log((q - 1.0) * (1.0 - x_pred) / x_pred, q) + x_pred
    while abs(x_pred - x) > 0.00001 :
        x_pred = x
        x = (y - entropy(x_pred, q))/log((q - 1.0) * (1.0 - x_pred) / x_pred, q) + x_pred
    return float(x)

In [32]:
sigma = 0.5 #écart type de la gaussienne selon x et y
var = sigma*sigma
m = 2
q = 2^m


mapping = matrix([[2, 1],\
                  [0, 3]])

# mapping de Gray

mapping = matrix([[2, 3],\
                  [0, 1]])


print("mapping :")
print(mapping)
print()

transition = [[0 for i in range(q)] for j in range(q)]
for x_i in range(2):
    for y_i in range(2):
        # cas 1
        x_j, y_j = (0,0)
        x_j_min, x_j_max = -oo, 0.5
        y_j_min, y_j_max = -oo, 0.5
        transition[mapping[x_i][y_i]][mapping[x_j][y_j]] = float(numerical_integral(lambda t : exp(((t - x_i)^2)/(-2*var)), x_j_min, x_j_max)[0] * numerical_integral(lambda t : exp(((t - y_i)^2)/(-2*var)), y_j_min, y_j_max)[0] / (2*pi*var))

        # cas 2
        x_j, y_j = (1,0)
        x_j_min, x_j_max = 0.5, +oo
        y_j_min, y_j_max = -oo, 0.5
        transition[mapping[x_i][y_i]][mapping[x_j][y_j]] = float(numerical_integral(lambda t : exp(((t - x_i)^2)/(-2*var)), x_j_min, x_j_max)[0] * numerical_integral(lambda t : exp(((t - y_i)^2)/(-2*var)), y_j_min, y_j_max)[0] / (2*pi*var))

        # cas 3
        x_j, y_j = (0,1)
        x_j_min, x_j_max = -oo, 0.5
        y_j_min, y_j_max = 0.5, +oo
        transition[mapping[x_i][y_i]][mapping[x_j][y_j]] = float(numerical_integral(lambda t : exp(((t - x_i)^2)/(-2*var)), x_j_min, x_j_max)[0] * numerical_integral(lambda t : exp(((t - y_i)^2)/(-2*var)), y_j_min, y_j_max)[0] / (2*pi*var))

        # cas 4
        x_j, y_j = (1,1)
        x_j_min, x_j_max = 0.5, +oo
        y_j_min, y_j_max = 0.5, +oo
        transition[mapping[x_i][y_i]][mapping[x_j][y_j]] = float(numerical_integral(lambda t : exp(((t - x_i)^2)/(-2*var)), x_j_min, x_j_max)[0] * numerical_integral(lambda t : exp(((t - y_i)^2)/(-2*var)), y_j_min, y_j_max)[0] / (2*pi*var))

print("transition :")
for line_transition in transition:
    print(line_transition)

mapping :
[2 3]
[0 1]

transition :
[0.707860981736997, 0.13348376433138823, 0.13348376433138823, 0.025171489600055073]
[0.13348376433138823, 0.707860981736997, 0.025171489600055073, 0.13348376433138823]
[0.13348376433138823, 0.025171489600055073, 0.707860981736997, 0.13348376433138823]
[0.025171489600055073, 0.13348376433138823, 0.13348376433138823, 0.707860981736997]


In [33]:
# Capacité du canal 4-QAM = QPSK

step = 0.05
C_max = 0
PX_max = [0.0]*4

PX = [0.0]*4
PX[0] = 0.0
while PX[0] <= 1.0:
    PX[1] = 0.0
    while PX[1] <= 1.0 - PX[0]:
        PX[2] = 0.0
        while PX[2] <= 1.0 - PX[0] - PX[1]:
            PX[3] = 1.0 - PX[0] - PX[1] - PX[2]
            
            PY = [0]*4
            PY[0] = float(sum([PX[x] * transition[x][0] for x in range(4)]))
            PY[1] = float(sum([PX[x] * transition[x][1] for x in range(4)]))
            PY[2] = float(sum([PX[x] * transition[x][2] for x in range(4)]))
            PY[3] = float(sum([PX[x] * transition[x][3] for x in range(4)]))
            
            HqY = -sum([(0 if PY[y] == 0 else float(PY[y] * log(PY[y], q))) for y in range(4)])
            HqY_X = sum([-PX[x]*sum([(0 if transition[x][y] == 0 else float(transition[x][y] * log(transition[x][y], q))) for y in range(4)]) for x in range(4)])
            C = HqY - HqY_X
            if C > C_max:
                C_max = C
                PX_max[0] = PX[0]
                PX_max[1] = PX[1]
                PX_max[2] = PX[2]
                PX_max[3] = PX[3]
            PX[2] += step
        PX[1] += step
    PX[0] += step

print("Capacité du canal QPSK = ", C_max)
print("Distribution emission : ", PX_max)

Capacité du canal QPSK =  0.368917232594362
Distribution emission :  [0.250000000000000, 0.250000000000000, 0.250000000000000, 0.250000000000000]


In [34]:
# Capacité du canal QPSK vu comme un canal q-aire symétrique

from sage.coding.code_bounds import entropy

p = 1.0 - transition[0][0]
Capacity = float(1.0 - entropy(p,q))
print("Capacité du canal q-aire symétrique = ", Capacity)

Capacité du canal q-aire symétrique =  0.3327511297890273


In [35]:
# Capacité du canal QPSK vu comme un canal binaire symétrique

# taux d'erreur binaire
p_bsc = 0
for i in range(q):
    for j in range(q):
        p_bsc += float(bin(i^^j).count("1") * transition[i][j] * (1.0/q) / m) # P(i) = 1/q  et m = nb bits par symbole

print("taux d'erreur binaire : ", p_bsc)
print("capacité du canal binaire symétrique : " , float(1 - entropy(p_bsc,2)))
print()

taux d'erreur binaire :  0.15865525393144328
capacité du canal binaire symétrique :  0.3689172325944913



In [37]:
sigma = 0.5 #écart type de la gaussienne selon x et y
var = sigma*sigma
m = 4
q = 2^m


'''mapping = matrix([[0, 8, 10, 2],\
                  [4, 12, 14, 6],
                  [5, 13, 15, 7],
                  [1, 9, 11, 3]])'''
mapping = matrix([[0, 1, 2, 3],\
                  [4, 5, 6, 7],
                  [8, 9, 10, 11],
                  [12, 13, 14, 15]])
print("mapping :")
print(mapping)
print()

transition = [[0 for i in range(q)] for j in range(q)]
for x_i in range(4):
    for y_i in range(4):
        for x_j in range(4):
            for y_j in range(4):
                if x_j == 0:
                    x_j_min = -oo
                    x_j_max = 0.5
                if x_j in [1,2]:
                    x_j_min = x_j - 0.5
                    x_j_max = x_j + 0.5
                if x_j == 3:
                    x_j_min = 2.5
                    x_j_max = +oo
                    
                if y_j == 0:
                    y_j_min = -oo
                    y_j_max = 0.5
                if y_j in [1,2]:
                    y_j_min = y_j - 0.5
                    y_j_max = y_j + 0.5
                if y_j == 3:
                    y_j_min = 2.5
                    y_j_max = +oo
                
                transition[mapping[x_i][y_i]][mapping[x_j][y_j]] = float(numerical_integral(lambda t : exp(((t - x_i)^2)/(-2*var)), x_j_min, x_j_max)[0] * numerical_integral(lambda t : exp(((t - y_i)^2)/(-2*var)), y_j_min, y_j_max)[0] / (2*pi*var))

print("transition :")
for line_transition in transition:
    for val in line_transition:
        print("%.3f" % round(val, 3), end=' ')
    print()
print()



mapping :
[ 0  1  2  3]
[ 4  5  6  7]
[ 8  9 10 11]
[12 13 14 15]

transition :
0.708 0.132 0.001 0.000 0.132 0.025 0.000 0.000 0.001 0.000 0.000 0.000 0.000 0.000 0.000 0.000 
0.133 0.574 0.132 0.001 0.025 0.107 0.025 0.000 0.000 0.001 0.000 0.000 0.000 0.000 0.000 0.000 
0.001 0.132 0.574 0.133 0.000 0.025 0.107 0.025 0.000 0.000 0.001 0.000 0.000 0.000 0.000 0.000 
0.000 0.001 0.132 0.708 0.000 0.000 0.025 0.132 0.000 0.000 0.000 0.001 0.000 0.000 0.000 0.000 
0.133 0.025 0.000 0.000 0.574 0.107 0.001 0.000 0.132 0.025 0.000 0.000 0.001 0.000 0.000 0.000 
0.025 0.108 0.025 0.000 0.108 0.466 0.107 0.001 0.025 0.107 0.025 0.000 0.000 0.001 0.000 0.000 
0.000 0.025 0.108 0.025 0.001 0.107 0.466 0.108 0.000 0.025 0.107 0.025 0.000 0.000 0.001 0.000 
0.000 0.000 0.025 0.133 0.000 0.001 0.107 0.574 0.000 0.000 0.025 0.132 0.000 0.000 0.000 0.001 
0.001 0.000 0.000 0.000 0.132 0.025 0.000 0.000 0.574 0.107 0.001 0.000 0.133 0.025 0.000 0.000 
0.000 0.001 0.000 0.000 0.025 0.107 0.025 0.000

In [40]:
# taux d'erreur q-aire
p_sc = float(1 - sum([transition[i][i] for i in range(16)])/16)

        
print("taux d'erreur q-aire : ", p_sc)
print("capacité du canal q-aire symétrique correspondant : " , float(1 - entropy(p_sc,q)))
print()

taux d'erreur q-aire :  0.4193299101943123
capacité du canal q-aire symétrique correspondant :  0.3451458471541585



In [41]:
# taux d'erreur binaire
p_bsc = 0
for i in range(16):
    for j in range(16):
        p_bsc += float(bin(i^^j).count("1") * transition[i][j] / 16)
p_bsc = float(p_bsc/4)

print("taux d'erreur binaire : ", p_bsc)
print("capacité du canal binaire symétrique correspondant : " , float(1 - entropy(p_bsc,2)))
print()

taux d'erreur binaire :  0.15831785108643281
capacité du canal binaire symétrique correspondant :  0.3697299099118353



In [43]:
step = 0.01
C_max = 0
pX_max = [0]*q
p0 = 0
while p0 <= 0.25:
    p1 = 0
    while p1 <= 0.25 - p0:
        p2 = float((1 - 4*p0 - 4*p1)/8)
        
        pX = [0]*16
        for x in [1,2]:
            for y in [1,2]:
                pX[mapping[x][y]] = p0
                
        for x in [0,3]:
            for y in [0,3]:
                pX[mapping[x][y]] = p1
        
        for x in [1,2]:
            for y in [0,3]:
                pX[mapping[x][y]] = p2
        for x in [0,3]:
            for y in [1,2]:
                pX[mapping[x][y]] = p2
          
        pY = [0]*q
        for j in range(16):
            pY[j] = float(sum([pX[i] * transition[i][j] for i in range(16)]))
            
        HqY = -sum([(0 if pY[j] == 0 else float(pY[j] * log(pY[j], 16))) for j in range(16)])
        HqY_X = sum([-pX[i]*sum([(0 if transition[i][j] == 0 else float(transition[i][j] * log(transition[i][j], 16))) for j in range(16)]) for i in range(16)])
        C = HqY - HqY_X
        if C > C_max:
            C_max = C
            for i in range(16):
                pX_max[i] = pX[i]
        
        
        p1 += step
    p0 += step

print("Capacité du canal 16-QAM = ", C_max)
print(pX_max)


Capacité du canal 16-QAM =  0.563350713883721
[0.130000000000000, 0.05000000000000002, 0.05000000000000002, 0.130000000000000, 0.05000000000000002, 0.0200000000000000, 0.0200000000000000, 0.05000000000000002, 0.05000000000000002, 0.0200000000000000, 0.0200000000000000, 0.05000000000000002, 0.130000000000000, 0.05000000000000002, 0.05000000000000002, 0.130000000000000]
